### Modelo de regresion de Red Neuronal para el analisis del precio promedio del aguacate.

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --- 1️⃣ Cargar datasets ya limpios y escalados ---
train_df = pd.read_csv("avocado_train_clean.csv")
test_df = pd.read_csv("avocado_test_clean.csv")

In [7]:
# --- 2️⃣ Separar variables predictoras y objetivo ---
X_train = train_df.drop(columns=["AveragePrice"])
y_train = train_df["AveragePrice"]

X_test = test_df.drop(columns=["AveragePrice"])
y_test = test_df["AveragePrice"]

print("✅ Datos cargados correctamente.")
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

✅ Datos cargados correctamente.
X_train shape: (14599, 67), X_test shape: (3650, 67)


In [8]:
print("Media de X_train (debería estar cerca de 0):")
print(X_train.mean().round(3).head())

Media de X_train (debería estar cerca de 0):
Total Volume   -0.0
4046            0.0
4225            0.0
4770           -0.0
Total Bags     -0.0
dtype: float64


In [ ]:
# --- 4️⃣ Entrenar la red neuronal ---
mlp = MLPRegressor(
    hidden_layer_sizes=(64, 32),  # dos capas ocultas
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42
)


mlp.fit(X_train, y_train)



In [ ]:
# --- 5️⃣ Evaluar desempeño ---
y_pred = mlp.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n📊 Resultados del modelo:")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")